In [1]:
import inspect
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
torch.cuda.device_count = lambda: len(list(os.environ["CUDA_VISIBLE_DEVICES"].split(",") if "CUDA_VISIBLE_DEVICES" in os.environ else []))
import importlib
from torch import nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lrs
import pytorch_lightning as pl
from transformers import LlamaForCausalLM, LlamaTokenizer
import random
from pandas import DataFrame
import os
import numpy as np
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftModel
import json
from unsloth import FastLanguageModel 

/home/tiger/.local/lib/python3.9/site-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/rec/movielens_unsloth",dtype = torch.bfloat16,load_in_4bit = True)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.109 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 9.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model = model.merge_and_unload()
model.save_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/rec/movielens_unsloth_merged")
tokenizer.save_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/rec/movielens_unsloth_merged")

In [3]:
model = LlamaForCausalLM.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/Llama-2-7b-hf", load_in_4bit=True)
tokenizer:LlamaTokenizer = LlamaTokenizer.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/Llama-2-7b-hf")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [10]:
ori_prompt = """You are a system that recommends movies based on viewing history. Please evaluate the similarity between each watch history in the candidate list and the single target watch history. Rate the similarity on a scale from 1 to 10 between , where 1 is not similar at all and 10 is very similar.

Please output the similarity ratings in JSON format. Here is the format:
{
"Watch History 1": score,
}

Candidate Watch History:
    Watch History 1: Young Frankenstein (1974) Full Metal Jacket (1987) American President, The (1995) Shining, The (1980) Somewhere in Time (1980) Aladdin (1992) Raising Arizona (1987) Room with a View, A (1986) Madness of King George, The (1994) Client, The (1994) 
Watch History 2: Some Like It Hot (1959) Third Man, The (1949) Princess Bride, The (1987) Bad Boys (1995) Stand by Me (1986) Evita (1996) Raiders of the Lost Ark (1981) Snow White and the Seven Dwarfs (1937) Breakfast at Tiffany's (1961) Godfather: Part II, The (1974) 
Watch History 3: American President, The (1995) Top Gun (1986) Calendar Girl (1993) Terminator 2: Judgment Day (1991) Local Hero (1983) Crying Game, The (1992) Great Race, The (1965) Silence of the Lambs, The (1991) Brazil (1985) Indiana Jones and the Last Crusade (1989) 
Watch History 4: Being There (1979) Groundhog Day (1993) Blues Brothers, The (1980) Amityville Curse, The (1990) What's Love Got to Do with It (1993) Sleepless in Seattle (1993) Great Escape, The (1963) Bram Stoker's Dracula (1992) Godfather: Part II, The (1974) Indiana Jones and the Last Crusade (1989) 
Watch History 5: Some Like It Hot (1959) Sound of Music, The (1965) Evil Dead II (1987) Crow, The (1994) Professional, The (1994) Dances with Wolves (1990) Jungle Book, The (1994) Field of Dreams (1989) Free Willy (1993) Jaws 3-D (1983) 
Watch History 6: Crow, The (1994) Santa Clause, The (1994) Crossing Guard, The (1995) Aladdin and the King of Thieves (1996) Bad Boys (1995) Terminator 2: Judgment Day (1991) Braveheart (1995) 12 Angry Men (1957) Home Alone (1990) Silence of the Lambs, The (1991) 
Watch History 7: Professional, The (1994) Toy Story (1995) Star Trek III: The Search for Spock (1984) Outbreak (1995) Aliens (1986) Forbidden Planet (1956) Ghost (1990) Rocket Man (1997) Nightmare Before Christmas, The (1993) Cowboy Way, The (1994) 
Watch History 8: Forrest Gump (1994) Fatal Instinct (1993) Field of Dreams (1989) Empire Strikes Back, The (1980) Addams Family Values (1993) Dead Poets Society (1989) Real Genius (1985) Terminator 2: Judgment Day (1991) Star Trek V: The Final Frontier (1989) Beavis and Butt-head Do America (1996) 
Watch History 9: Snow White and the Seven Dwarfs (1937) GoodFellas (1990) Star Trek VI: The Undiscovered Country (1991) Silence of the Lambs, The (1991) Red Rock West (1992) Right Stuff, The (1983) Star Trek III: The Search for Spock (1984) Remains of the Day, The (1993) Rudy (1993) Citizen Kane (1941) 
Watch History 10: Village of the Damned (1995) What's Eating Gilbert Grape (1993) Some Like It Hot (1959) Sound of Music, The (1965) Evil Dead II (1987) Crow, The (1994) Professional, The (1994) Dances with Wolves (1990) Jungle Book, The (1994) Field of Dreams (1989) 


Target Watch History:
The Fugitive Die Hard Speed Blade Runner True Romance Aliens Terminator 2: Judgment Day Clear and Present Danger Executive Decision Devil in a Blue Dress

The output should only contain the JSON object with similarity scores, without any additional text. Output:
"""

In [4]:
prompt_template = """[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{usr_prompt} [/INST]"""

system_prompt = """You are a system that recommends movies based on viewing history. Please evaluate the similarity between each watch history in the candidate list and the single target watch history. Rate the similarity on a scale from 1 to 10 between , where 1 is not similar at all and 10 is very similar.

Please output the similarity ratings in JSON format. Here is the format:
{
"Watch History 1": score,
}"""

usr_prompt = """
Candidate Watch History:
Watch History 1: Young Frankenstein (1974) Full Metal Jacket (1987) American President, The (1995) Shining, The (1980) Somewhere in Time (1980) Aladdin (1992) Raising Arizona (1987) Room with a View, A (1986) Madness of King George, The (1994) Client, The (1994) 
Watch History 2: Some Like It Hot (1959) Third Man, The (1949) Princess Bride, The (1987) Bad Boys (1995) Stand by Me (1986) Evita (1996) Raiders of the Lost Ark (1981) Snow White and the Seven Dwarfs (1937) Breakfast at Tiffany's (1961) Godfather: Part II, The (1974) 
Watch History 3: American President, The (1995) Top Gun (1986) Calendar Girl (1993) Terminator 2: Judgment Day (1991) Local Hero (1983) Crying Game, The (1992) Great Race, The (1965) Silence of the Lambs, The (1991) Brazil (1985) Indiana Jones and the Last Crusade (1989) 
Watch History 4: Being There (1979) Groundhog Day (1993) Blues Brothers, The (1980) Amityville Curse, The (1990) What's Love Got to Do with It (1993) Sleepless in Seattle (1993) Great Escape, The (1963) Bram Stoker's Dracula (1992) Godfather: Part II, The (1974) Indiana Jones and the Last Crusade (1989) 
Watch History 5: Some Like It Hot (1959) Sound of Music, The (1965) Evil Dead II (1987) Crow, The (1994) Professional, The (1994) Dances with Wolves (1990) Jungle Book, The (1994) Field of Dreams (1989) Free Willy (1993) Jaws 3-D (1983) 
Watch History 6: Crow, The (1994) Santa Clause, The (1994) Crossing Guard, The (1995) Aladdin and the King of Thieves (1996) Bad Boys (1995) Terminator 2: Judgment Day (1991) Braveheart (1995) 12 Angry Men (1957) Home Alone (1990) Silence of the Lambs, The (1991) 
Watch History 7: Professional, The (1994) Toy Story (1995) Star Trek III: The Search for Spock (1984) Outbreak (1995) Aliens (1986) Forbidden Planet (1956) Ghost (1990) Rocket Man (1997) Nightmare Before Christmas, The (1993) Cowboy Way, The (1994) 
Watch History 8: Forrest Gump (1994) Fatal Instinct (1993) Field of Dreams (1989) Empire Strikes Back, The (1980) Addams Family Values (1993) Dead Poets Society (1989) Real Genius (1985) Terminator 2: Judgment Day (1991) Star Trek V: The Final Frontier (1989) Beavis and Butt-head Do America (1996) 
Watch History 9: Snow White and the Seven Dwarfs (1937) GoodFellas (1990) Star Trek VI: The Undiscovered Country (1991) Silence of the Lambs, The (1991) Red Rock West (1992) Right Stuff, The (1983) Star Trek III: The Search for Spock (1984) Remains of the Day, The (1993) Rudy (1993) Citizen Kane (1941) 
Watch History 10: Village of the Damned (1995) What's Eating Gilbert Grape (1993) Some Like It Hot (1959) Sound of Music, The (1965) Evil Dead II (1987) Crow, The (1994) Professional, The (1994) Dances with Wolves (1990) Jungle Book, The (1994) Field of Dreams (1989) 


Target Watch History:
The Fugitive Die Hard Speed Blade Runner True Romance Aliens Terminator 2: Judgment Day Clear and Present Danger Executive Decision Devil in a Blue Dress

The output should only contain the JSON object with similarity scores, without any additional text. Output:"""

prompt = prompt_template.format_map({"system_prompt":system_prompt, "usr_prompt":usr_prompt})

In [10]:
FastLanguageModel.for_inference(model)

In [5]:
print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}


In [8]:
print(tokenizer)
tokenizer.padding_side="left"

LlamaTokenizer(name_or_path='/mnt/bn/data-tns-live-llm/leon/datasets/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [11]:
prompt = ori_prompt

In [13]:
input = tokenizer(prompt, return_tensors="pt")
try: output = model.generate(input["input_ids"].cuda(), temperature=0.1, max_new_tokens=128, repetition_penalty=1.1).cpu()[0]
except: output = model.generate(input["input_ids"], temperature=0.1, max_new_tokens=128, repetition_penalty=1.1).cpu()[0]
output = tokenizer.decode(output)
print(output)

<s> You are a system that recommends movies based on viewing history. Please evaluate the similarity between each watch history in the candidate list and the single target watch history. Rate the similarity on a scale from 1 to 10 between , where 1 is not similar at all and 10 is very similar.

Please output the similarity ratings in JSON format. Here is the format:
{
"Watch History 1": score,
}

Candidate Watch History:
    Watch History 1: Young Frankenstein (1974) Full Metal Jacket (1987) American President, The (1995) Shining, The (1980) Somewhere in Time (1980) Aladdin (1992) Raising Arizona (1987) Room with a View, A (1986) Madness of King George, The (1994) Client, The (1994) 
Watch History 2: Some Like It Hot (1959) Third Man, The (1949) Princess Bride, The (1987) Bad Boys (1995) Stand by Me (1986) Evita (1996) Raiders of the Lost Ark (1981) Snow White and the Seven Dwarfs (1937) Breakfast at Tiffany's (1961) Godfather: Part II, The (1974) 
Watch History 3: American President, 

: 

In [23]:
import re
# output = json.loads("{"+output.split("Output:")[1].split("{")[1].split("}")[0]+"}")
sorted_items = sorted(output.items(), key=lambda item: item[1], reverse=True)
print(sorted_items)
# 获取分数最高的前5个键
top_5_idx = [int(re.findall(r'\d+', item[0])[0])-1 for item in sorted_items[:5]]
print(top_5_idx)

[('Watch History 1', 10), ('Watch History 2', 9), ('Watch History 3', 8), ('Watch History 4', 7), ('Watch History 5', 6), ('Watch History 6', 5), ('Watch History 7', 4), ('Watch History 8', 3), ('Watch History 9', 2), ('Watch History 10', 1)]
[0, 1, 2, 3, 4]


In [18]:
print(len(tokenizer("""{"Watch History 1": 9.0,"Watch History 2": 8.0,"Watch History 3": 8.0,"Watch History 4": 8.0,"Watch History 5": 8.0,"Watch History 6": 8.0,"Watch History 7": 8.0,"Watch History 8": 8.0,"Watch History 9": 8.0,"Watch History 10": 8.0}""")["input_ids"]))

103
